# Generating HTML for Plotly utilizing the Python API

Plotly has open sourced their javascript code! However, the functionality in their Python API has seen very minimal changes.  The only addition has been an inclusion for offline mode for ipython notebooks. I started with a tutorial found on reddit which easily outlines how to get this working in IPython, linked here:
 - https://www.reddit.com/r/IPython/comments/3tibc8/tip_on_how_to_run_plotly_examples_in_offline_mode/

After getting this example working, I delved into the offline source code to figure out how the HTML is being generated and displayed.  Fortunately, everything in the offline library has the tools we need to get the html we need, the javascript behind it, and with a little searching, eliminating the linking to the plotly servers (when you don't want to upload your data!)

All of the code we're modifying is found here:
 - https://github.com/plotly/plotly.py/tree/master/plotly/offline

## Let's look at the IPython plotting function

The main plotting function, iplot is found in offline.py.  I've annotated the code below into sections to breakdown what this function is doing, and what we can pull from it.

In [2]:
def iplot(figure_or_data, show_link=True, link_text='Export to plot.ly',
          validate=True):
    
    ########################################
    # Documentation
    ########################################
    
    # Here is the description of the function, and an example how to use it
    """
    Draw plotly graphs inside an IPython notebook without
    connecting to an external server.
    To save the chart to Plotly Cloud or Plotly Enterprise, use
    `plotly.plotly.iplot`.
    To embed an image of the chart, use `plotly.image.ishow`.
    figure_or_data -- a plotly.graph_objs.Figure or plotly.graph_objs.Data or
                      dict or list that describes a Plotly graph.
                      See https://plot.ly/python/ for examples of
                      graph descriptions.
    Keyword arguments:
    show_link (default=True) -- display a link in the bottom-right corner of
                                of the chart that will export the chart to
                                Plotly Cloud or Plotly Enterprise
    link_text (default='Export to plot.ly') -- the text of export link
    validate (default=True) -- validate that all of the keys in the figure
                               are valid? omit if your version of plotly.js
                               has become outdated with your version of
                               graph_reference.json or if you need to include
                               extra, unnecessary keys in your figure.
    Example:
    ```
    from plotly.offline import init_notebook_mode, iplot
    init_notebook_mode()
    iplot([{'x': [1, 2, 3], 'y': [5, 2, 7]}])
    ```
    """
    ########################################
    # IPython Check
    ########################################
    
    # These are some functions to ensure the user is using ipython notebook
    
    if not __PLOTLY_OFFLINE_INITIALIZED:
        raise PlotlyError('\n'.join([
            'Plotly Offline mode has not been initialized in this notebook. '
            'Run: ',
            '',
            'import plotly',
            'plotly.offline.init_notebook_mode() '
            '# run at the start of every ipython notebook',
        ]))
    if not tools._ipython_imported:
        raise ImportError('`iplot` can only run inside an IPython Notebook.')

        
    ########################################
    # Setup Parameters
    ########################################    
    
    # This sets up default values to use for the plot
    
    from IPython.display import HTML, display
    figure = tools.return_figure_from_figure_or_data(figure_or_data, validate)

    width = figure.get('layout', {}).get('width', '100%')
    height = figure.get('layout', {}).get('height', 525)
    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    plotdivid = uuid.uuid4()
    jdata = json.dumps(figure.get('data', []), cls=utils.PlotlyJSONEncoder)
    jlayout = json.dumps(figure.get('layout', {}), cls=utils.PlotlyJSONEncoder)
    
    ########################################
    # Config the link for exporting to plotly below plot
    ########################################
    
    # Creates the hyper link to export the data and open in plotly's website
    
    config = {}
    config['showLink'] = show_link
    config['linkText'] = link_text
    jconfig = json.dumps(config)

    plotly_platform_url = session.get_session_config().get('plotly_domain',
                                                           'https://plot.ly')
    if (plotly_platform_url != 'https://plot.ly' and
            link_text == 'Export to plot.ly'):

        link_domain = plotly_platform_url\
            .replace('https://', '')\
            .replace('http://', '')
        link_text = link_text.replace('plot.ly', link_domain)
        
    ########################################
    # Not Sure what this does?
    ########################################
    
    # I think this creates the html to display the plotly url?
    
    display(HTML(
        '<script type="text/javascript">'
        'window.PLOTLYENV=window.PLOTLYENV || {};'
        'window.PLOTLYENV.BASE_URL="' + plotly_platform_url + '";'
        '</script>'
    ))
    
    ########################################
    # Generate Javascript
    ########################################
    
    # Create a string of javascript from the id, data, layout, and configuration
    script = '\n'.join([
        'Plotly.plot("{id}", {data}, {layout}, {config}).then(function() {{',
        '    $(".{id}.loading").remove();',
        '}})'
    ]).format(id=plotdivid,
              data=jdata,
              layout=jlayout,
              config=jconfig)
    ########################################
    # Display HTML
    ########################################
    
    #using ipython's display and HTML functions, we pass in the html and javascript to display in the notebook
    
    display(HTML(''
                 '<div class="{id} loading" style="color: rgb(50,50,50);">'
                 'Drawing...</div>'
                 '<div id="{id}" style="height: {height}; width: {width};" '
                 'class="plotly-graph-div">'
                 '</div>'
                 '<script type="text/javascript">'
                 '{script}'
                 '</script>'
                 ''.format(id=plotdivid, script=script,
                           height=height, width=width)))

This function has everything we need to display how to generate the plot in the ipython notebook.  It generates html and javascript, which is then embedded into the ipython notebook. All we need to do now is get this html and javascript and embed it into our own webpages!

## Alright, let's get everything we need to run everything offline

### Getting the Plotly Javascript

Plotly has given us two easy ways to get the plotly javascript.  They have a nice easy function you can call from offline, get_plotlyjs, listed below.  The file itself can also just be found in the offline repo listed at the beginning

In [1]:
def get_plotlyjs():
    path = os.path.join('offline', 'plotly.min.js')
    plotlyjs = resource_string('plotly', path).decode('utf-8')
    return plotlyjs


### Removing external link functionality

If you are familiar with the plotly tooltip, there is a link at the top that saves your plot in the cloud so you can work on it there.  I'm not currently sure if this data is stored and made public or not, so on the safe side, we may want to remove this functionality from any plots we make.  In order to do this, we can replace the link that is used in the javascript. The link to search for is:
 - https://plot.ly
 
The save icon links to https://plot.ly/external, with some string concatination in the script.  With some greater knowledge of javascript beyond what I have, you can create some dynamic links if you want to open a page with just the image.  For now, I simply replaced it with a null string, which tries to open /external, which should not resolve.  You can also replace it with 0.0.0.0 so it just looks locally.

### Redefining the iplot function

We can leverage the iplot function, which has all of the functionality we need to generate the html we need.  Here is a modified version of it:

In [10]:
from plotly import session, tools, utils
import uuid
import json



def new_iplot(figure_or_data, show_link=True, link_text='Export to plot.ly',
          validate=True):

    figure = tools.return_figure_from_figure_or_data(figure_or_data, validate)

    width = figure.get('layout', {}).get('width', '100%')
    height = figure.get('layout', {}).get('height', 525)
    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    plotdivid = uuid.uuid4()
    jdata = json.dumps(figure.get('data', []), cls=utils.PlotlyJSONEncoder)
    jlayout = json.dumps(figure.get('layout', {}), cls=utils.PlotlyJSONEncoder)

    config = {}
    config['showLink'] = show_link
    config['linkText'] = link_text
    jconfig = json.dumps(config)

    plotly_platform_url = session.get_session_config().get('plotly_domain',
                                                           'https://plot.ly')
    if (plotly_platform_url != 'https://plot.ly' and
            link_text == 'Export to plot.ly'):

        link_domain = plotly_platform_url\
            .replace('https://', '')\
            .replace('http://', '')
        link_text = link_text.replace('plot.ly', link_domain)


    script = '\n'.join([
        'Plotly.plot("{id}", {data}, {layout}, {config}).then(function() {{',
        '    $(".{id}.loading").remove();',
        '}})'
    ]).format(id=plotdivid,
              data=jdata,
              layout=jlayout,
              config=jconfig)

    html="""<div class="{id} loading" style="color: rgb(50,50,50);">
                 Drawing...</div>
                 <div id="{id}" style="height: {height}; width: {width};" 
                 class="plotly-graph-div">
                 </div>
                 <script type="text/javascript">
                 {script}
                 </script>
                 """.format(id=plotdivid, script=script,
                           height=height, width=width)

    return html

The main changes made to this are:
 - Removed ipython notebook checks
 - Removed html displays, and simply generated html

We now have all the tools we need!  One final note: When calling the new_iplot function, you will want to set show_link to False, to remove the second export link in the app.  Additionally, you may want to change the urls listed in the function to point the link wherever you want.

### Example on how to generate html

All we have to do now is to pass in either data or a figure into our new_iplot function to generate the html! We can use any of the plotly functions.  Let's look at a scatter plot:

In [11]:
from plotly.graph_objs import Scatter
import numpy as np

N = 10
random_x = np.random.randn(N)
random_y = np.random.randn(N)

# Create a trace
trace = Scatter(
    x = random_x,
    y = random_y,
    mode = 'markers'
)

data = [trace]

#call our modified function to generate html
html=new_iplot(data,show_link=False)

In [13]:
print html

<div class="bb03b202-5da1-4da4-8fe5-f90dfc838bbe loading" style="color: rgb(50,50,50);">
                 Drawing...</div>
                 <div id="bb03b202-5da1-4da4-8fe5-f90dfc838bbe" style="height: 525; width: 100%;" 
                 class="plotly-graph-div">
                 </div>
                 <script type="text/javascript">
                 Plotly.plot("bb03b202-5da1-4da4-8fe5-f90dfc838bbe", [{"y": [0.9911873318040711, -0.8729646106022118, 1.821836891443137, 0.07754671642149655, -0.48396671505725236, 1.2024273667318313, -0.6562320595893218, -1.2591301953284428, -1.9911418562002885, -0.03485012790324974], "x": [-1.5901231648274856, 0.02613254715607766, 0.10113028405315724, 0.4435984469100101, 0.39515719918122527, -1.8652722239664923, 0.5660972261975588, 0.9969284454158368, 0.5638210591224765, 0.1315570489898769], "type": "scatter", "mode": "markers"}], {}, {"linkText": "Export to plot.ly", "showLink": false}).then(function() {
    $(".bb03b202-5da1-4da4-8fe5-f90dfc838bbe.loa

### Now we've got our html!

All we need to do now is embed this into our html, as well as the javascript file we grabbed earlier! Now you have a plotly plot in a webpage, without going through their servers!

## Utilizing Data Spyre 

I recently gave a talk on how to setup a data spyre server, and integrate bokeh and pygal plots into it, which can be found here:
- https://github.com/pm8k/dataspyre_tutorial

With our new plotly html, we can embed it into our data spyre app

In [ ]:
# import web app modules
from spyre import server
import cherrypy
cherrypy.config.update({'server.socket_host': '0.0.0.0'})

import os
import pandas as pd
import numpy as np

from plotly.graph_objs import Scatter

from db import DB

# Here I saved the modified functions into a file called plotly_modified.py, and imported it
import plotly_modified as pm


##########################################################################################################
# Data Spyre Class and Functions
##########################################################################################################

class plotly_test(server.App):

    title='Plotly Test'

    inputs = [  {"type":'text',
                    "label": 'Random Input',
                    "key": 'number',
                    "value":'10',
                    "action_id":'update_data'},
                ]

    controls = [{   "type" : "hidden",
                    "label" : "Refresh",
                    "id" : "update_data"}]


    outputs = [
                {   "type" : "html",
                    "id" : "getHTML_Plotly",
                    "control_id" : "update_data",
                    "on_page_load" : True}]


    ######################################################################################################

    def getHTML_Plotly(self,params):

        N = int(params['number'])
        random_x = np.random.randn(N)
        random_y = np.random.randn(N)

        # Create a trace
        trace = Scatter(
            x = random_x,
            y = random_y,
            mode = 'markers'
        )

        data = [trace]

        #call our modified function to generate html
        html=pm.modified_iplot(data,show_link=False)
        return html
    
    def getCustomJS(self):
        # plotlyjs.js is the javascript file from the plotly repo
        with open ("plotlyjs.js", "r") as myfile:
            data=myfile.read().replace('\n', '')
        return data

##########################################################################################################
#define main function to create the server

if __name__ == '__main__':
    app=plotly_test()
    app.launch(port=9092)

There are two main compenents here to generate our plotly plot:
 - We need a function to get the html.  In this example, we generate the data using plotly's function, pass it into our iplot function, and return the html from it.
 - Similar to Bokeh, we can add custom javascript.  For this I read in the javascript file into a string, and return it.
 
We can now launch our data spyre app, and we have data loaded in it!

## Thanks for reading!